In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

import sys

import pandas as pd
from from_root import from_root
from ib_insync import IB, LimitOrder, util
from loguru import logger

from src.utils import Vars, get_pickle, place_orders, quick_pf

util.startLoop()

# Set the root
ROOT = from_root()
_vars = Vars(MARKET)
PORT = port = _vars.PORT
DATAPATH = ROOT / 'data' / MARKET

# set vscode path
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>

In [ ]:
df_tgt = get_pickle(DATAPATH / "df_naked_targets.pkl")
df = df_tgt.dropna(subset=['expPrice'])

OPT_COLS = _vars.OPT_COLS[0]

In [ ]:
# REMOVE PORTFOLIO SYMBOLS FROM TARGET
# ====================================

# Get the portfolio
with IB().connect(port=PORT) as ib:
    df_pf = quick_pf(ib)

# Remove targets which are already in the portfolio
if df_pf:
    df = df[~df.symbol.isin(set(df_pf.symbol))]

In [ ]:
# REMOVE OPEN ORDER SYMBOLS
# =========================

with IB().connect(port=PORT) as ib:
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    # ib.sleep(0.3)
    trades = ib.trades()

if trades:
    all_trades_df = (util.df(t.contract for t in trades).join(
        util.df(t.orderStatus
                for t in trades)).join(util.df(t.order for t in trades),
                                        lsuffix="_"))

    all_trades_df.rename({"lastTradeDateOrContractMonth": "expiry"},
                            axis="columns",
                            inplace=True)
    trades_cols = ["conId", "symbol", "secType", "expiry",
        "strike", "right", "orderId", "permId", "action",
        "totalQuantity", "lmtPrice", "status",]

    dfo = all_trades_df[trades_cols]
    df_openords = dfo[all_trades_df.status.isin(_vars. ACTIVE_STATUS)]

else:
    df_openords = None
    

try:
    if df_openords:  # There are some open orders
        df = df[~df.symbol.isin(set(df_openords.symbol))]
except ValueError:
    pass

In [ ]:
# Make (contract, order) tuple
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]

In [ ]:
len(cos)

In [ ]:
df[['symbol', 'strike', 'undPrice', 'right', 'expiry', 'dte', 'margin', 'optPrice', 'expPrice']].dropna(subset=['optPrice'])